In [ ]:
from wayne_utils import load_data, save_data
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from config import _ROOT_PATH, R3_ALL_METHODS
from utils import Record_result, Check_result, split_merge
from eval.R3.Syntax_Accuracy import SA_Batch
from eval.R3.R3_Execute_Accuracy import R3_EA_Batch, R3_ExecuteAccuracy
from eval.R3.Comprehension_Accuracy import CA_Batch
from eval.R3.My_postprocess import post_execute_result
from eval.R3.My_Execute_Accuracy import My_EA_Batch
from eval.SpCQL.CA_Metric import Neo4j_ComprehensionAccuracy
from eval.R3.My_Exact_Set_Match import ExactSetMatch

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/n3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
allow_list = {
    "EM": True,
    "SA": True,
    "CA": True,
    "R3_EA": True,
    "EA": True
}

In [ ]:

gold_query_list = load_data( os.path.join( _ROOT_PATH, "data/R3/R3_gold_query_list.json" ), "json")
gold_result_list = load_data( os.path.join( _ROOT_PATH, "data/R3/R3_gold_result_list.pickle"), "pickle")
for method in ["FT"]:          # ["MyMethod"]:      # R3_ALL_METHODS.keys(): #  #
    print( method )
    for sub_method in ["GLM4-9B"]:      # R3_ALL_METHODS[method].keys():  # ["Chinese-Mistral-7B-Instruct-v0.1"]:     #  
        # if methods[method][sub_method]:
        
        whole_method_name = method +"-" + sub_method if sub_method != "" else method
        sub_method = None if sub_method == "" else sub_method
        print( whole_method_name )
        
        if method == "Gold":
            predict_query_path = os.path.join( _ROOT_PATH, "data/R3/R3_gold_query_list.json" )
            predict_result_path = os.path.join( _ROOT_PATH, "data/R3/R3_gold_result_list.pickle")
        else:
            predict_query_path = os.path.join( _ROOT_PATH, f"inter_data/R3/query_list/{whole_method_name}_query_list.json" )
            predict_result_path = os.path.join( _ROOT_PATH, f"inter_data/R3/execute_result_list/{whole_method_name}_execute_results.pickle")
        
        if os.path.exists( predict_query_path ) and os.path.exists( predict_result_path ):
            print( f"{whole_method_name}的query list和exec result都存在，可以进行eval")
            
            predict_query_list = load_data( predict_query_path, "json")
            if isinstance(predict_query_list, dict):
                predict_query_list, _ = split_merge( predict_query_list )
            predict_result_list = load_data( predict_result_path, "pickle")
            

            if allow_list[ "EM" ] and Check_result( "EM", method, dataset = "R3", sub_method = sub_method, overwrite = False):
                EM, EM_list = ExactSetMatch( gold_query_list, predict_query_list )
                Record_result( "EM", EM, method, "R3", sub_method)
            if allow_list[ "CA" ] and Check_result( "CA", method, dataset = "R3", sub_method = sub_method, overwrite = False):
                # CA, CA_list = CA_Batch( predict_query_list, gold_query_list, model_type="roberta-large")
                CA = Neo4j_ComprehensionAccuracy( predict_query_list, gold_query_list)
                Record_result( "CA", CA, method, "R3", sub_method)
            
            if allow_list["SA"] or allow_list["R3_EA"] or allow_list["EA"]:
                SA, SA_list, ER = SA_Batch( predict_result_list )                       # query语义错误
            
            if allow_list[ "SA" ] and Check_result( "SA", method, dataset = "R3", sub_method = sub_method, overwrite = False):
                Record_result( "SA", SA, method, "R3", sub_method)
                Record_result( "ER", ER, method, "R3", sub_method)

            predict_result_list = post_execute_result( predict_result_list )
            gold_result_list = post_execute_result( gold_result_list )
            if allow_list[ "R3_EA" ] and Check_result( "R3_EA", method, dataset = "R3", sub_method = sub_method, overwrite = False):
                R3_EA, R3_IEA = R3_EA_Batch( predict_result_list, gold_result_list, SA_list )
                Record_result( "R3_EA", R3_EA, method, "R3", sub_method)
                Record_result( "R3_IEA", R3_IEA, method, "R3", sub_method)
            
            if allow_list[ "EA" ] and Check_result( "EA", method,  dataset = "R3", sub_method = sub_method, overwrite = False):
                EA, IEA, EEA, EA_list, EEA_list = My_EA_Batch( predict_result_list, gold_result_list, SA_list)     # 结果精确
                Record_result( "EA", EA, method, "R3", sub_method)
                Record_result( "EEA", EEA, method, "R3", sub_method)
                Record_result( "IEA", IEA, method, "R3", sub_method)
        else:
            print( f"路径不存在{predict_query_path}或者{predict_result_path}")
            """"""

FT
FT-GLM4-9B
FT-GLM4-9B的query list和exec result都存在，可以进行eval
不存在记录FT-GLM4-9B-EM，允许重写。
新的记录：FTGLM4-9B的EM = 16.34768740031898
不存在记录FT-GLM4-9B-CA，允许重写。


/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/n3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Bert Scoring: 1254it [00:24, 50.32it/s]


新的记录：FTGLM4-9B的CA = 45.8207132116252


SA evaluating...: 100%|██████████| 1254/1254 [00:00<00:00, 1423067.43it/s]


不存在记录FT-GLM4-9B-SA，允许重写。
新的记录：FTGLM4-9B的SA = 80.622009569378
新的记录：FTGLM4-9B的ER = 20.015948963317385
不存在记录FT-GLM4-9B-N3_EA，允许重写。


N3 EA evaluating...:   0%|          | 0/1254 [00:00<?, ?it/s]/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/n3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
N3 EA evaluating...:   0%|          | 1/1254 [00:02<59:58,  2.87s/it]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.569 seconds.
Prefix dict has been built successfully.
N3 EA evaluating...: 100%|██████████| 1254/1254 [00:24<00:00, 51.94it/s] 


新的记录：FTGLM4-9B的N3_EA = 55.57716479887038
新的记录：FTGLM4-9B的N3_IEA = 68.93547443895496
不存在记录FT-GLM4-9B-EA，允许重写。


After evaluating: 100%|██████████| 1254/1254 [00:00<00:00, 5587.36it/s]

新的记录：FTGLM4-9B的EA = 39.473684210526315
新的记录：FTGLM4-9B的EEA = 44.896331738437
新的记录：FTGLM4-9B的IEA = 48.961424332344215
